In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set_style("whitegrid")

In [11]:
TREE = pd.read_csv('../Data/WA_TREE.csv')
print(len(TREE))
TREE.columns

/var/folders/mm/t_p68_6506v3jg0m1f58zkv40000gn/T/ipykernel_45123/2074286447.py:1: DtypeWarning: Columns (77,159) have mixed types. Specify dtype option on import or set low_memory=False.
  TREE = pd.read_csv('../Data/WA_TREE.csv')


504956


Index(['CN', 'PLT_CN', 'PREV_TRE_CN', 'INVYR', 'STATECD', 'UNITCD', 'COUNTYCD',
       'PLOT', 'SUBP', 'TREE',
       ...
       'VOLCSNET_BARK', 'DRYBIO_STEM', 'DRYBIO_STEM_BARK', 'DRYBIO_STUMP_BARK',
       'DRYBIO_BOLE_BARK', 'DRYBIO_BRANCH', 'DRYBIO_FOLIAGE',
       'DRYBIO_SAWLOG_BARK', 'PREV_ACTUALHT_FLD', 'PREV_HT_FLD'],
      dtype='object', length=197)

$N = 504,956$ 
- Now we can remove trees that are in plots that have not burned in the last 24 years.  
- To do this I will use **PLOT_FIRES_GIS**, which was created by intersecting the last 24 years of fire data from the NIFC with our plot latitude and longitude.  *We are introducing our first uncertainty here- the plot latitude and longitude have been 'fuzzed'.*

In [36]:
FIREPLOTS = pd.read_csv('../Data/WA_PLOT_FIRE_PAIRS.csv')
print(len(FIREPLOTS))
print(FIREPLOTS.columns)


7177
Index(['CN', 'SRV_CN', 'CTY_CN', 'PREV_PLT_CN', 'INVYR', 'STATECD', 'UNITCD',
       'COUNTYCD', 'PLOT', 'PLOT_STATUS_CD', 'PLOT_NONSAMPLE_REASN_CD',
       'MEASYEAR', 'MEASMON', 'MEASDAY', 'REMPER', 'KINDCD', 'DESIGNCD',
       'RDDISTCD', 'WATERCD', 'LAT', 'LON', 'ELEV', 'GROW_TYP_CD',
       'MORT_TYP_CD', 'P2PANEL', 'P3PANEL', 'ECOSUBCD', 'CONGCD', 'MANUAL',
       'KINDCD_NC', 'QA_STATUS', 'CREATED_DATE', 'MODIFIED_DATE',
       'MICROPLOT_LOC', 'DECLINATION', 'EMAP_HEX', 'SAMP_METHOD_CD',
       'SUBP_EXAMINE_CD', 'MACRO_BREAKPOINT_DIA', 'INTENSITY', 'CYCLE',
       'SUBCYCLE', 'ECO_UNIT_PNW', 'TOPO_POSITION_PNW',
       'NF_SAMPLING_STATUS_CD', 'NF_PLOT_STATUS_CD',
       'NF_PLOT_NONSAMPLE_REASN_CD', 'P2VEG_SAMPLING_STATUS_CD',
       'P2VEG_SAMPLING_LEVEL_DETAIL_CD', 'INVASIVE_SAMPLING_STATUS_CD',
       'INVASIVE_SPECIMEN_RULE_CD', 'DESIGNCD_P2A', 'MANUAL_DB', 'SUBPANEL',
       'CONDCHNGCD_RMRS', 'FUTFORCD_RMRS', 'MANUAL_NCRS', 'MANUAL_NERS',
       'MANUAL_RMRS', 'PAC

**There are 7177 plots*fire pairs that have burned in the last 24 years.**
- Now we filter TREE by using these plots.  

In [41]:
TREE_IN_BURNED_PLOTS = TREE[TREE['PLOT'].isin(FIREPLOTS['PLOT'])]
len(TREE_IN_BURNED_PLOTS)

128183

$N = 128,183$
- This is still a lot of trees that appear in plots that have burned.  
- Now we need to filter for PLOTs that have been measured on either side of a burn
    - These measurements years are shown in the **WA_PLOT.csv** dataset in the *MEASYEAR* column

- **PLOT_FIRES_GIS** contains fire, plot pairs.  
    - Lets go through these pairs, referencing WA_PLOT to check if the measurement years are on either side of the fire year
        - This will filter **PLOT_FIRES_GIS** and will add the 2 years measured as columns to **PLOT_FIRES_GIS**.

In [40]:
PLOTS = pd.read_csv('../Data/WA_PLOT.csv')
PLOTS.columns

Index(['CN', 'SRV_CN', 'CTY_CN', 'PREV_PLT_CN', 'INVYR', 'STATECD', 'UNITCD',
       'COUNTYCD', 'PLOT', 'PLOT_STATUS_CD', 'PLOT_NONSAMPLE_REASN_CD',
       'MEASYEAR', 'MEASMON', 'MEASDAY', 'REMPER', 'KINDCD', 'DESIGNCD',
       'RDDISTCD', 'WATERCD', 'LAT', 'LON', 'ELEV', 'GROW_TYP_CD',
       'MORT_TYP_CD', 'P2PANEL', 'P3PANEL', 'ECOSUBCD', 'CONGCD', 'MANUAL',
       'KINDCD_NC', 'QA_STATUS', 'CREATED_DATE', 'MODIFIED_DATE',
       'MICROPLOT_LOC', 'DECLINATION', 'EMAP_HEX', 'SAMP_METHOD_CD',
       'SUBP_EXAMINE_CD', 'MACRO_BREAKPOINT_DIA', 'INTENSITY', 'CYCLE',
       'SUBCYCLE', 'ECO_UNIT_PNW', 'TOPO_POSITION_PNW',
       'NF_SAMPLING_STATUS_CD', 'NF_PLOT_STATUS_CD',
       'NF_PLOT_NONSAMPLE_REASN_CD', 'P2VEG_SAMPLING_STATUS_CD',
       'P2VEG_SAMPLING_LEVEL_DETAIL_CD', 'INVASIVE_SAMPLING_STATUS_CD',
       'INVASIVE_SPECIMEN_RULE_CD', 'DESIGNCD_P2A', 'MANUAL_DB', 'SUBPANEL',
       'CONDCHNGCD_RMRS', 'FUTFORCD_RMRS', 'MANUAL_NCRS', 'MANUAL_NERS',
       'MANUAL_RMRS', 'PAC_ISLA

In [53]:
FIREPLOTS.sample(3)

,CN,SRV_CN,CTY_CN,PREV_PLT_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,PLOT_STATUS_CD,...,PREV_PLOT_STATUS_CD_RMRS,REUSECD1,REUSECD2,REUSECD3,GRND_LYR_SAMPLING_STATUS_CD,GRND_LYR_SAMPLING_METHOD_CD,IRWINID,FIRE_YEAR,INCIDENT,GIS_ACRES
6738,290008196489998,273493076489998,60010497,2.239889e+13,2015,53,6,9,71039,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1951,Forks,32697.910
2292,30852567020004,30431009020004,92010497,2.239830e+13,2012,53,8,47,71067,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,Carlton Complex,251965.300
6029,412222872489998,384841470489998,88010497,2.295609e+13,2017,53,9,25,74107,2,...,NaN,NaN,NaN,NaN,NaN,NaN,{A7034F3F-54D7-4C38-9E80-937F85490E42},2016,Lower Crab Creek,6244.313


In [63]:
FIREPLOTS_MEAS = FIREPLOTS.copy()
FIREPLOTS_MEAS['MEASYEAR1,MEASYEAR2'] = None
# This will just add MEASYEAR1 and MEASYEAR2
#After we will filter for rows where the fire occurs in between
nums_of_measurements = np.zeros(len(FIREPLOTS))
for index,row in FIREPLOTS_MEAS.iterrows():
    plot = row['PLOT']
    unitcd = row['UNITCD']
    countycd = row['COUNTYCD']
    
    uniqueplot = PLOTS[(PLOTS['PLOT']==plot) & (PLOTS['UNITCD'] == unitcd) & (PLOTS['COUNTYCD']==countycd)]
    if len(uniqueplot) == 2:
        FIREPLOTS_MEAS['MEASYEAR1'] = min(uniqueplot.MEASYEAR)
        FIREPLOTS_MEAS['MEASYEAR2'] = max(uniqueplot.MEASYEAR)


In [76]:
FIREPLOTS_FIRE_SANDWICH = FIREPLOTS_MEAS[(FIREPLOTS_MEAS['FIRE_YEAR']>FIREPLOTS_MEAS['MEASYEAR1']) & (FIREPLOTS_MEAS['FIRE_YEAR']<FIREPLOTS_MEAS['MEASYEAR2'])]
print(len(FIREPLOTS_FIRE_SANDWICH))
print(FIREPLOTS_FIRE_SANDWICH['PLOT'].nunique())

1729
539


**IF** I have done this properly there are 1729 (think hardy) fire, plot pairs that are sandwiched between two measurements.  These pairs are on 539 plots.  \
Let me check a couple rows..

In [83]:
FIREPLOTS_FIRE_SANDWICH[['MEASYEAR1','FIRE_YEAR','MEASYEAR2']].sample(10)

,MEASYEAR1,FIRE_YEAR,MEASYEAR2
139,2005,2007,2015
1265,2005,2006,2015
5322,2005,2012,2015
1275,2005,2011,2015
3504,2005,2006,2015
2740,2005,2014,2015
2393,2005,2014,2015
656,2005,2012,2015
4277,2005,2008,2015
2806,2005,2014,2015


**BANG!** \
Look at how those fires are sandwiched

OK.

I am claiming that we use this **FIREPLOTS_FIRE_SANDWICH** dataset to filter our trees.  This will give us our final dataset.  

PLEASE double check my work and logic behind the filtering.  